# DeepLearning / Convolutional networks

## Lecture

다른 위치에 있어도 같은 텍스트/이미지 즉, 같은 대상/피쳐가 있을수 있는데, 공간정보등을 인지 안하고 일일이 학습시키는 것은 비효율적
어느 위치에 있던, 어느 컨텍스트에 있던 고양이는 고양이님. 사실상 같음. "Statistical Invariance"
![L4-1]( ./images/L4-1a-motiv.png)
![L4-1]( ./images/L4-1b-motiv.png)

ConvNet은 Weight Sharing을 통해서 분별하는 접근.  
중요한 개념으로 , feature map, pooling, patch/filter/kernel, stride 등이 있음  
자세한건 강의참고.

![Conv]( ./images/L4-2a-Conv.png)
![Conv]( ./images/L4-2c-Conv.png)
![Conv]( ./images/L4-2d-Conv.png)
![Conv]( ./images/L4-2g-Conv.png)
![Conv]( ./images/L4-2e-Conv.png)
![Conv]( ./images/L4-2f-Conv.png)


## Assignmnet 4 

Previously in 2_fullyconnected.ipynb and 3_regularization.ipynb, we trained fully connected networks to classify notMNIST characters.
The goal of this assignment is make the neural network convolutional.


** References:** 
- [Arn-O](https://github.com/Arn-O/udacity-deep-learning/blob/master/4_convolutions.ipynb)  

** Other ConvNet examples:**
- [aymericdamien](https://github.com/aymericdamien/TensorFlow-Examples/blob/master/notebooks/3_NeuralNetworks/convolutional_network.ipynb)
- [terryum](https://github.com/terryum/TensorFlow_Exercises/blob/master/4a_CNN_MNIST_160517.ipynb)
- [tensorflow tutorial/ codeOnWeb](https://codeonweb.com/entry/f50e23df-0f23-4e56-95a6-efb9981716f7)

In [2]:
# These are all the modules we'll be using later. Make sure you can import them before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [3]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save
    
    print ('Training Set', train_dataset.shape, train_labels.shape)
    print ('Validation set', valid_dataset.shape, valid_labels.shape)
    print ('Test set', test_dataset.shape, test_labels.shape)

Training Set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


리포맷  
Reformat into a TensorFlow-friendly shape: 
- convolutions need the image dadta formatted as a cube(width X height X # of channels )
- labels as float 1-hot encodings

In [4]:
image_size =  28
num_labels = 10
num_channels = 1 #gray scale

import numpy as np

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size, image_size, num_channels)).astype(np.float32)
    labels = (np.arange(num_labels) == labels[:, None]).astype(np.float32)
    return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)

print ('Training Set', train_dataset.shape, train_labels.shape)
print ('Validation Set', valid_dataset.shape, train_labels.shape)
print ('Test set', test_dataset.shape, test_labels.shape)

Training Set (200000, 28, 28, 1) (200000, 10)
Validation Set (10000, 28, 28, 1) (200000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [5]:
def accuracy(predictions, labels):
    return( 100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels,1))/predictions.shape[0]  )

콘볼루션 레이어(2 layer)와 fully connected layer 구현:

Let's build a small network with two convolutional layers, followed by one fully connected layer.  
Convolutional networks are more expensive computationally,  
so we'll limit its depth and number of fully connected nodes.

In [6]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

    #input data
    tf_train_dataset = tf.placeholder(tf.float32, shape = (batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape = (batch_size, num_labels) )
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    #Variables 
    layer1_weights = tf.Variable(tf.truncated_normal( [patch_size, patch_size, num_channels, depth], stddev = 0.1) )
    layer1_biases = tf.Variable(tf.zeros([depth]))
    
    layer2_weights = tf.Variable(tf.truncated_normal( [patch_size, patch_size, depth, depth], stddev=0.1) )
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    
    layer3_weights = tf.Variable(tf.truncated_normal( [image_size/4*image_size/4*depth, num_hidden], stddev = 0.1) )
    layer3_biases = tf.Variable(tf.constant(1.0,shape = [num_hidden] ))
    
    layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1 ))
    layer4_biases = tf.Variable(tf.constant(1.0, shape = [num_labels]))
    
    # Model
    def model(data):
        conv = tf.nn.conv2d(data, layer1_weights, [1,2,2,1], padding = 'SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        conv = tf.nn.conv2d(hidden, layer2_weights,[1,2,2,1], padding = 'SAME' )
        hidden = tf.nn.relu(conv + layer2_biases)

        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1]*shape[2]*shape[3] ])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        
        return tf.matmul(hidden, layer4_weights) + layer4_biases
    

    # Training Computation:
    logits = model(tf_train_dataset)
    loss= tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
    # Optimizer
    optimizer = tf.train.GradientDescentOptimizer(.05).minimize(loss)
    
    # Predictions for the training, validation, and test data
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

** :Execute **

In [7]:
num_steps = 1001

with tf.Session(graph = graph ) as session:
    tf.initialize_all_variables().run()
    print ('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset+ batch_size), :]
        feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels}
        _,l,predictions = session.run([optimizer, loss, train_prediction], feed_dict= feed_dict)
        if (step%50 ==0):
            print ('Minibatch loss at step: %d: %f' % (step, l))
            print ('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print ('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))

    print('Test Accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step: 0: 3.171521
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%
Minibatch loss at step: 50: 1.578632
Minibatch accuracy: 50.0%
Validation accuracy: 47.5%
Minibatch loss at step: 100: 1.237140
Minibatch accuracy: 62.5%
Validation accuracy: 70.1%
Minibatch loss at step: 150: 0.672105
Minibatch accuracy: 75.0%
Validation accuracy: 75.6%
Minibatch loss at step: 200: 0.801610
Minibatch accuracy: 75.0%
Validation accuracy: 77.5%
Minibatch loss at step: 250: 1.090486
Minibatch accuracy: 75.0%
Validation accuracy: 79.0%
Minibatch loss at step: 300: 0.544587
Minibatch accuracy: 75.0%
Validation accuracy: 79.7%
Minibatch loss at step: 350: 0.766908
Minibatch accuracy: 81.2%
Validation accuracy: 78.0%
Minibatch loss at step: 400: 0.373723
Minibatch accuracy: 93.8%
Validation accuracy: 78.0%
Minibatch loss at step: 450: 0.735488
Minibatch accuracy: 81.2%
Validation accuracy: 79.0%
Minibatch loss at step: 500: 0.786490
Minibatch accuracy: 87.5%
Validation accura

## Problem 1

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation(i.e. nn.max_pool()) of stride 2 and kernel size2.

** * 요약: **
위 과정은 stride 값이 2인 covolution 을 통해 차원 축소를 하였음.  
이를 maxpooling , kernel size, stride=2 의 조건으로 재 시행.


In [14]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():
    
    # Input Data
    tf_train_dataset = tf.placeholder(tf.float32, shape = (batch_size, image_size, image_size, num_channels) )
    tf_train_labels = tf.placeholder(tf.float32, shape = (batch_size, num_labels) )
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variable
    layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev = 0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    
    layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev = 0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape = [depth]))
    
    layer3_weights = tf.Variable(tf.truncated_normal([image_size/4 * image_size/4 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape = [num_hidden]))
    
    layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape = [num_labels])) 
    
    def model(data):
        conv1 = tf.nn.conv2d(data, layer1_weights, [1,1,1,1], padding = 'SAME')
        bias1 = tf.nn.relu(conv1 + layer1_biases)
        pool1 = tf.nn.max_pool(bias1, [1,2,2,1], [1,2,2,1], padding = 'SAME')

        conv2 = tf.nn.conv2d(pool1, layer2_weights, [1,1,1,1], padding = 'SAME')
        bias2 = tf.nn.relu(conv2 + layer2_biases)
        pool2 = tf.nn.max_pool(bias2, [1,2,2,1], [1,2,2,1], padding = 'SAME')

        shape = pool2.get_shape().as_list()
        reshape = tf.reshape(pool2, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)

        return tf.matmul(hidden, layer4_weights) + layer4_biases
    
    # Training Computation
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
    # Optimizer
    optimizer = tf.train.GradientDescentOptimizer(.05).minimize(loss)
    
    # Prediction
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))
    


Running the Session:

In [15]:
num_steps = 1001

with tf.Session(graph = graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels: batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict= feed_dict)
        
        if(step%50 == 0):
            print ('Minibatch loss at step %d: %f' % (step, l))
            print ('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print ('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels ))
        
    print('Test Accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))
    

Initialized
Minibatch loss at step 0: 3.173182
Minibatch accuracy: 12.5%
Validation accuracy: 11.7%
Minibatch loss at step 50: 2.246892
Minibatch accuracy: 12.5%
Validation accuracy: 25.6%
Minibatch loss at step 100: 1.581455
Minibatch accuracy: 37.5%
Validation accuracy: 53.0%
Minibatch loss at step 150: 0.845521
Minibatch accuracy: 81.2%
Validation accuracy: 70.5%
Minibatch loss at step 200: 1.000629
Minibatch accuracy: 75.0%
Validation accuracy: 75.6%
Minibatch loss at step 250: 1.362105
Minibatch accuracy: 62.5%
Validation accuracy: 77.7%
Minibatch loss at step 300: 0.510090
Minibatch accuracy: 81.2%
Validation accuracy: 80.3%
Minibatch loss at step 350: 0.842779
Minibatch accuracy: 87.5%
Validation accuracy: 78.0%
Minibatch loss at step 400: 0.255478
Minibatch accuracy: 100.0%
Validation accuracy: 80.3%
Minibatch loss at step 450: 0.617679
Minibatch accuracy: 87.5%
Validation accuracy: 78.8%
Minibatch loss at step 500: 0.815000
Minibatch accuracy: 81.2%
Validation accuracy: 80.0%


## Problem 2
Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture,  
adding Dropout, and/or adding learning rate decay.

** *요약: Dropout, learning rate deacy를 써보자. **

** - NO Dropout &  Learning Rate Decay  Version- **

In [21]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():
    
    #Input Data
    tf_train_dataset = tf.placeholder(tf.float32, shape = [batch_size, image_size, image_size, num_channels] )
    tf_train_labels = tf.placeholder(tf.float32, shape = [batch_size, num_labels] )

    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    #Variables 
    layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    
    layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev = .1 ))
    layer2_biases = tf.Variable(tf.constant(1.0 , shape = [depth]))

    size3 = ( (image_size - patch_size + 1 )/2 - patch_size + 1 )/2    
    
    layer3_weights = tf.Variable(tf.truncated_normal([size3 * size3 * depth, num_hidden], stddev= .1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape = [num_hidden]))
    
    layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape = [num_labels]))
    
    def model(data):
        # C1 input 28 X 28
        conv1 = tf.nn.conv2d(data, layer1_weights, [1,1,1,1], padding = 'VALID')
        bias1 = tf.nn.relu(conv1+ layer1_biases)
        
        # S2 input 24 X 24
        pool2 = tf.nn.avg_pool(bias1, [1,2,2,1], [1,2,2,1], padding = 'VALID')
        
        # C3 input 12 X 12
        conv3 = tf.nn.conv2d(pool2, layer2_weights, [1,1,1,1], padding = 'VALID')
        bias3 = tf.nn.relu(conv3+ layer2_biases)
        
        # S4 input 8 X 8 
        pool4 = tf.nn.avg_pool(bias3, [1,2,2,1], [1,2,2,1], padding = 'VALID')
        
        # F6 input 4 X 4
        shape = pool4.get_shape().as_list()
        reshape = tf.reshape(pool4, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases) 
        
        return tf.matmul(hidden, layer4_weights) + layer4_biases
    
    # Training computation
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
    # Optimizer
    optimizer = tf.train.GradientDescentOptimizer(.05).minimize(loss)
    
    # Predictions for the training, validation, and test data
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))
    

Running the Session:

In [23]:
num_steps = 2001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        feed_dict = {tf_train_dataset: batch_data, tf_train_labels : batch_labels}
        _,l,predictions = session.run([optimizer, loss, train_prediction], feed_dict = feed_dict)
        
        if(step%100 == 0):
            print ('Minibatch loss at step %d: %f' % (step, l))
            print ('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print ('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels ))
        
    print('Test Accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.483344
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 100: 1.319547
Minibatch accuracy: 50.0%
Validation accuracy: 60.1%
Minibatch loss at step 200: 0.844818
Minibatch accuracy: 81.2%
Validation accuracy: 72.5%
Minibatch loss at step 300: 0.658113
Minibatch accuracy: 87.5%
Validation accuracy: 75.2%
Minibatch loss at step 400: 0.396682
Minibatch accuracy: 87.5%
Validation accuracy: 77.3%
Minibatch loss at step 500: 0.763901
Minibatch accuracy: 81.2%
Validation accuracy: 78.7%
Minibatch loss at step 600: 0.263028
Minibatch accuracy: 93.8%
Validation accuracy: 79.8%
Minibatch loss at step 700: 0.962759
Minibatch accuracy: 68.8%
Validation accuracy: 80.4%
Minibatch loss at step 800: 0.605703
Minibatch accuracy: 75.0%
Validation accuracy: 81.0%
Minibatch loss at step 900: 0.568709
Minibatch accuracy: 87.5%
Validation accuracy: 82.1%
Minibatch loss at step 1000: 0.623930
Minibatch accuracy: 81.2%
Validation accuracy: 81.9%


** - Dropout &  Learning Rate Decay  Version- **

In [35]:
batch_size = 16
patch_size = 5
depth = 16

num_hidden = 64
beta_regul = 1e-3
drop_out = .5

graph = tf.Graph()

with graph.as_default():
    
    #Input data
    tf_train_dataset = tf.placeholder(tf.float32, shape = (batch_size, image_size, image_size, num_channels))
    tf_train_labels= tf.placeholder(tf.float32, shape=(batch_size, num_labels) )

    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    global_step = tf.Variable(0)
    
    #Variables
    layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev= 0.1) )
    layer1_biases = tf.Variable(tf.zeros([depth]))
    
    layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape = [depth]))
    
    size3= ((image_size-patch_size +1)/2 - patch_size +1)/2 
    
    layer3_weights = tf.Variable(tf.truncated_normal([size3*size3*depth, num_hidden], stddev=-.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape = [num_hidden]))
    
    layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_hidden], stddev =0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    
    layer5_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev = 0.1))
    layer5_biases  = tf.Variable(tf.constant(1.0, shape = [num_labels]))
    
    # Model
    def model(data, keep_prob):
        # C1 input 28 X 28
        conv1 = tf.nn.conv2d(data, layer1_weights, [1,1,1,1], padding = 'VALID')
        bias1 = tf.nn.relu(conv1+layer1_biases)
        
        # S2 input 24 X 24
        pool2 = tf.nn.avg_pool(bias1, [1,2,2,1], [1,2,2,1], padding = 'VALID')
        
        # C3 input 12 X 12
        conv3 = tf.nn.conv2d(pool2, layer2_weights, [1,1,1,1], padding = 'VALID')
        bias3 = tf.nn.relu(conv3 + layer2_biases)
        
        # S4 input 8 X 8 
        pool4 = tf.nn.avg_pool(bias3, [1,2,2,1],[1,2,2,1], padding = 'VALID')
        
        # F5 input 4 X 4
        shape = pool4.get_shape().as_list()
        reshape = tf.reshape(pool4, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden5 = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        
        # F6 
        drop5 = tf.nn.dropout(hidden5, keep_prob)
        hidden6 = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        drop6 = tf.nn.dropout(hidden6, keep_prob)
        return tf.matmul(drop6, layer5_weights) + layer5_biases
    
    # Training computation
    logits = model(tf_train_dataset, drop_out)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
    # Optimizer
    learning_rate = tf.train.exponential_decay(0.05, global_step, 1000, 0.85, staircase = True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step= global_step)
    
    # predictions for the training, validation, and test data
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset, 1.0))
    test_prediction = tf.nn.softmax(model(tf_test_dataset, 1.0))

In [38]:
num_steps = 5001

with tf.Session(graph = graph ) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    
    for step in range(num_steps):
        offset = (step*batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset: (offset+batch_size), :,:,:]
        batch_labels = train_labels[offset: (offset+batch_size), :]

        feed_dict = {tf_train_dataset: batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict = feed_dict)
        
        if(step %500 ==0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels)) 

Initialized
Minibatch loss at step 0: 3.309491
Minibatch accuracy: 6.2%
Validation accuracy: 12.3%
Minibatch loss at step 500: 1.146324
Minibatch accuracy: 75.0%
Validation accuracy: 75.2%
Minibatch loss at step 1000: 0.657059
Minibatch accuracy: 81.2%
Validation accuracy: 79.5%
Minibatch loss at step 1500: 0.888884
Minibatch accuracy: 68.8%
Validation accuracy: 81.2%
Minibatch loss at step 2000: 0.178765
Minibatch accuracy: 100.0%
Validation accuracy: 82.2%
Minibatch loss at step 2500: 0.587984
Minibatch accuracy: 81.2%
Validation accuracy: 83.2%
Minibatch loss at step 3000: 0.985930
Minibatch accuracy: 81.2%
Validation accuracy: 83.7%
Minibatch loss at step 3500: 0.880156
Minibatch accuracy: 81.2%
Validation accuracy: 83.7%
Minibatch loss at step 4000: 0.548153
Minibatch accuracy: 81.2%
Validation accuracy: 84.7%
Minibatch loss at step 4500: 0.874191
Minibatch accuracy: 75.0%
Validation accuracy: 85.1%
Minibatch loss at step 5000: 1.302955
Minibatch accuracy: 62.5%
Validation accurac